In [1]:
import os
import time
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger
from utils.data import SleepDataLoader
from models import HeartRateNetwork

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sub_ids = [x.split('_')[0] for x in os.listdir('data')]
data_loader = SleepDataLoader(sub_ids)
train_loader = data_loader.train(batch_size=1200)
val_loader = data_loader.val(batch_size=1200)
test_loader = data_loader.test(batch_size=1200)

In [4]:
for x, y in train_loader:
    print(x.shape, y.shape)
    print(x.dtype, type(y))
    break

torch.Size([1200, 256]) torch.Size([1200])
torch.float32 <class 'torch.Tensor'>


In [5]:
callbacks = [EarlyStopping(monitor='val_BinaryF1Score', patience=3, mode='max', verbose=True),
             ModelCheckpoint(monitor='val_BinaryF1Score', mode='max')]

run_name = 'MESA_' + time.strftime("%Y%m%d-%H%M%S")
model = HeartRateNetwork(lr=0.0001, weight_decay=0.25)

In [6]:
logger = WandbLogger(project='sleep-ihr', name=run_name, log_model='all')
# logger.experiment.config.update(args)

trainer = Trainer(accelerator='gpu', devices=1, max_epochs=1, logger=logger, callbacks=callbacks)
trainer.fit(model, train_loader, val_loader)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ares1510. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name            | Type             | Params
-----------------------------------------------------
0 | input_conv      | Conv1d           | 16    
1 | cnn_blocks      | Sequential       | 27.2 K
2 | embedding_layer | Linear           | 262 K 
3 | dilated_blocks  | Sequential       | 1.2 M 
4 | output_conv     | Conv1d           | 516   
5 | criterion       | CrossEntropyLoss | 0     
6 | train_metrics   | MetricCollection | 0   

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/osamamohammed/.conda/envs/osamahar/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/osamamohammed/.conda/envs/osamahar/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   4%|▍         | 19/443 [00:00<00:20, 20.37it/s, v_num=5rlz]

/home/osamamohammed/.conda/envs/osamahar/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


Epoch 0: 100%|██████████| 443/443 [00:13<00:00, 32.27it/s, v_num=5rlz]

RuntimeError: Early stopping conditioned on metric `val_BinaryF1Score` which is not available. Pass in or modify your `EarlyStopping` callback to use any of the following: `train_loss`, `train_loss_step`, `val_loss`, `val_loss_epoch`, `val_MulticlassAccuracy`, `val_MulticlassAUROC`, `val_MulticlassF1Score`, `train_loss_epoch`, `train_MulticlassAccuracy`, `train_MulticlassAUROC`, `train_MulticlassF1Score`